# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LORA
* Model: distilbert-base-uncased
* Evaluation approach: Hugging Face Trainer class
* Fine-tuning dataset:fancyzhx/amazon_polarity

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [36]:
#The following code in the three sections (Loading and Evaluating the model, Performing Parameter-Efficient Fine-Tuning 
#and Performing Inference with a PEFT Model)was generated with the assistance from VOX chat gpt-35-turbo-16k (8-11 Dec-2024),
#GPT-4 from OpenAI (8-11 Dec-2024), and MicrosoftCopilot(8-11 Dec-2024)

#Code from the UDACITY GenAI fundamentals was also taken as reference to complete the code.
#Specific errors solutions are specified with the URL in the steps that were needed

#*****************************************

#Load a pretrained HF Distillbert model



from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2, #Binary classification, 1 = positive, 0 = negative
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1},

)

print(model)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [37]:
#Tokenize data set
from datasets import load_dataset

# Tokenize data set
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Load the train and test splits of the dataset
dataset = load_dataset("fancyzhx/amazon_polarity")

# Create a smaller subset for train and test- Copilot helped
subset_size = min(5000, len(dataset["train"]), len(dataset["test"]))
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(subset_size))
small_test_dataset = dataset["test"].shuffle(seed=42).select(range(subset_size))

print("Before tokenization column names:", small_train_dataset.column_names) #Printed collumns before tokenization to check whats happening

print(small_train_dataset[0])

Before tokenization column names: ['label', 'title', 'content']
{'label': 0, 'title': 'Anyone who likes this better than the Pekinpah is a moron.', 'content': "All the pretty people in this film. Even the Rudy character played by Michael Madsen. This is adapted from a Jim Thompson novel for cryin' out loud! These are supposed to be marginal characters, not fashion models. Though McQueen and McGraw were attractive (but check out McQueen's crummy prison haircut) they were believable in the role. Baldwin and Bassinger seem like movie stars trying to act like hard cases. Action wise, the robbery scene in the Pekinpah version was about 100 times more exciting and suspenseful than anything in this re-make."}


In [38]:
# Tokenize data. Copilot helped
def tokenize_function(example):
    return tokenizer(example["content"], padding="max_length", truncation=True)

tokenized_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = small_test_dataset.map(tokenize_function, batched=True)

print("After tokenization column name:", tokenized_train_dataset.column_names) #Printed collumns after tokenization to check whats happening




After tokenization column name: ['label', 'title', 'content', 'input_ids', 'attention_mask']


In [40]:
#https://stackoverflow.com/questions/78031519/how-to-resolve-valueerror-you-should-supply-an-encoding-or-a-list-of-encodings
train_lora = tokenized_train_dataset.rename_column('label', 'labels')
test_lora = tokenized_test_dataset.rename_column('label', 'labels')

#Remove unnecessary columns- Copilot helped since the error of not providing inputs-id keept coming 
train_lora = train_lora.remove_columns(['title', 'content'])
test_lora = test_lora.remove_columns(['title', 'content'])

# print("After renaming and removing unnecessary columns:", train_lora.column_names) #Printed collumns after removing to check whats happening

print(f"Number of training samples: {len(train_lora)}")
print(f"Number of test samples: {len(test_lora)}")

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_lora, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_lora, batch_size=16)

# Check the first batch
# for batch in train_dataloader:
#     print(batch)
#     break

Number of training samples: 5000
Number of test samples: 5000


In [41]:
#Define the compute_metrics for evaluating accuracy of the foundational model prior training. Copilot and pdf support document from George Stamos helped
import numpy as np
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir=".",
        learning_rate=2e-3,
        per_device_train_batch_size=1, # Reduce batch size for not collapsing the memory
        per_device_eval_batch_size=1,
        num_train_epochs=0,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=train_lora,
    eval_dataset=test_lora,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer,
    padding="max_length"),
    compute_metrics=compute_metrics,
)


In [42]:
#Evaluate the model prior training
print("Evaluating the model before fine-tuning:...")
pre_fine_tuning = trainer.evaluate()
print(pre_fine_tuning)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6975525617599487, 'eval_accuracy': 0.4774, 'eval_runtime': 92.3941, 'eval_samples_per_second': 54.116, 'eval_steps_per_second': 54.116}


# Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [43]:
#Creating a PEFT Config https://huggingface.co/docs/peft/package_reference/config
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification, TaskType
import bitsandbytes as bnb

#Copilot helped with the correct target_modules
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, #Task type for sequence classification
    r=8,
    lora_alpha=32,
    inference_mode=False, 
    target_modules=["q_lin", "v_lin"], #focusing solely on the linear transformations related to the query and passage encoding steps. As copilot suggestions, since using also (k_lin) was causing and error of not providng inputs-id
    bias="none",
    lora_dropout=0.01,
)


# Apply LoRA to the model
lora_model = get_peft_model(model, lora_config)


#8-bit optimizers:
adam = bnb.optim.Adam(lora_model.parameters(), 
                      lr=0.001, 
                      betas=(0.9, 0.995), 
                      optim_bits=32, 
                      percentile_clipping=5)

print(lora_model)


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(
                  in_features=768, out_features=768, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.01, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=8, bias=Fal

In [44]:
#Train the PEFT model with Copilot help
#Prepare Data Loaders: Create data loaders for training and evaluation:

from transformers import Trainer, TrainingArguments, default_data_collator, DataCollatorWithPadding, TrainerCallback, TrainerState, TrainerControl


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    loss = eval_pred[0]
    predictions = np.argmax(predictions, axis=1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy, "eval_loss": loss}

# Custom callback to log evaluation metrics. Copilot suggestion for the purpose to perform additional actions during training or evaluation at specific points.
class CustomCallback(TrainerCallback):
    def on_evaluate(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        metrics = kwargs.get("metrics", {})
        print(f"Evaluation metrics at step {state.global_step}: {metrics}")

In [45]:
# Configure the Trainer
# for load_best_model_at_end=False to correct eval loss error taken from stackoverflow blog
#remove_unused_columns=False,https://discuss.huggingface.co/t/indexerror-invalid-key-16-is-out-of-bounds-for-size-0/14298/25
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir=".",
        learning_rate=2e-3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=False,
        remove_unused_columns=False, #https://stackoverflow.com/questions/76796304/invalid-key-409862-is-out-of-bounds-for-size-0
       
    ),
    train_dataset=train_lora, 
    eval_dataset=test_lora,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length"),
    compute_metrics=compute_metrics,
    callbacks=[CustomCallback],  # Add the custom callback here
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.273215,0.893400
2,0.316800,0.285620,0.895200
3,0.316800,0.350202,0.904600


Evaluation metrics at step 313: {'eval_loss': 0.2732151746749878, 'eval_accuracy': 0.8934, 'eval_runtime': 85.5653, 'eval_samples_per_second': 58.435, 'eval_steps_per_second': 3.658, 'epoch': 1.0}


Checkpoint destination directory ./checkpoint-313 already exists and is non-empty.Saving will proceed but saved results may be invalid.


Evaluation metrics at step 626: {'eval_loss': 0.2856197655200958, 'eval_accuracy': 0.8952, 'eval_runtime': 85.5968, 'eval_samples_per_second': 58.413, 'eval_steps_per_second': 3.657, 'epoch': 2.0}


Checkpoint destination directory ./checkpoint-626 already exists and is non-empty.Saving will proceed but saved results may be invalid.


Evaluation metrics at step 939: {'eval_loss': 0.3502024710178375, 'eval_accuracy': 0.9046, 'eval_runtime': 85.451, 'eval_samples_per_second': 58.513, 'eval_steps_per_second': 3.663, 'epoch': 3.0}


Checkpoint destination directory ./checkpoint-939 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=939, training_loss=0.25343988193109773, metrics={'train_runtime': 793.6188, 'train_samples_per_second': 18.901, 'train_steps_per_second': 1.183, 'total_flos': 2021091102720000.0, 'train_loss': 0.25343988193109773, 'epoch': 3.0})

In [46]:
#Checking Trainable Parameters of a PEFT Model
lora_model.print_trainable_parameters()

# Evaluate the model after fine-tuning
print("Evaluating the model after fine-tuning...")

post_finetune_eval = trainer.evaluate()

print(post_finetune_eval)


trainable params: 1,331,716 || all params: 67,694,596 || trainable%: 1.967241225577297
Evaluating the model after fine-tuning...


Evaluation metrics at step 939: {'eval_loss': 0.3502024710178375, 'eval_accuracy': 0.9046, 'eval_runtime': 85.759, 'eval_samples_per_second': 58.303, 'eval_steps_per_second': 3.65, 'epoch': 3.0}
{'eval_loss': 0.3502024710178375, 'eval_accuracy': 0.9046, 'eval_runtime': 85.759, 'eval_samples_per_second': 58.303, 'eval_steps_per_second': 3.65, 'epoch': 3.0}


In [47]:
#Save the PEFT model
lora_model.save_pretrained("distilbert-base-uncased-model-lora_model")

#confirmed that the model was saved. Copilot helped to check
import os

save_directory = "distilbert-base-uncased-model-lora_model"
file_list = os.listdir(save_directory)

if len(file_list) > 0:
    print("Model saved successfully.")
else:
    print("Model saving failed.")

Model saved successfully.


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [48]:
#Load the Pretrained Model

from peft import get_peft_model, AutoPeftModelForSequenceClassification

# Load the saved PEFT model
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("distilbert-base-uncased-model-lora_model")


lora_model.eval()

print(lora_model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(
                  in_features=768, out_features=768, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.01, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=8, bias=Fal

In [49]:
# Define compute_metrics. Copilot helped
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [52]:
# Trainer for PEFT model
trainer_peft = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir=".",
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        remove_unused_columns=False,
    ),
    eval_dataset=test_lora,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length"),
    compute_metrics=compute_metrics,
)

In [53]:
#Evaluate the Performance of the Trained PEFT Model. The accuracy did not improve dramatically and it might need more epochs:
print("Evaluating the loaded fine-tuned model...")
results_peft = trainer_peft.evaluate()
print("Evaluating the loaded fine-tuned Lora_model Results:", results_peft)

Evaluating the loaded fine-tuned model...


Evaluating the loaded fine-tuned Lora_model Results: {'eval_loss': 0.3502024710178375, 'eval_accuracy': 0.9046, 'eval_runtime': 97.9864, 'eval_samples_per_second': 51.027, 'eval_steps_per_second': 51.027}


In [49]:
# Assuming results from prior evaluation are stored in `results`
# Compare with the foundation model results
print("Prior-training model Results:", results)

Prior-training model Results: {'eval_loss': 0.6942567825317383, 'eval_accuracy': 0.4852, 'eval_runtime': 92.9459, 'eval_samples_per_second': 53.795, 'eval_steps_per_second': 53.795}
